In [ ]:
#imports
import mne
import glob
import os
import re
import numpy as np
from itertools import chain
from matplotlib import pyplot as plt
from mne.time_frequency import psd_multitaper
from mne.decoding import LinearModel
from mne.decoding import get_coef
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import pandas as pd

In [ ]:
train = pd.read_excel('/Users/ilyamikheev/Downloads/ages.xlsx', header=1)

In [ ]:
from lightgbm import LGBMClassifier
X_train, X_test, y_train, y_test = train_test_split(train.drop(columns=['ID', 'AGE', 'COND', 'N1P2', 'OLD', 'P1_LAT', 'P2_LAT', 'N1_LAT', 'N2_LAT']), train['OLD'], test_size=0.2, random_state=42)
lgb = LGBMClassifier(objective='multiclass')
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)
lgb.fit(X_train, y_train)
predictions = lgb.predict_proba(X_test.to_numpy())

In [ ]:
import lightgbm
lightgbm.plot_importance(lgb, max_num_features=50, figsize=(16,9))
plt.savefig('imp_amp.png')

In [ ]:
import pandas
from scipy.stats import mstats

Data = pandas.read_csv("/Users/ilyamikheev/Downloads/ages_td.csv")

In [ ]:
import numpy as np
c_names = ['P1_AMP', 'N1_AMP', 'P2_AMP', 'N2_AMP', 'P1_LAT', 'P2_LAT', 'N1_LAT', 'N2_LAT']
m_v = np.zeros([8,6])
res = np.zeros([8,2])
for i, name in enumerate(c_names):
    Col_1 = Data[Data['OLD']=='A'][name]
    Col_2 = Data[Data['OLD']=='B'][name]
    Col_3 = Data[Data['OLD']=='C'][name]
    m_v[i,0], m_v[i,1], m_v[i,2], m_v[i,3], m_v[i,4], m_v[i,5] = Col_1.median(), Col_2.median(), Col_3.median(), Col_1.std(), Col_2.std(), Col_3.std()
    res[i,0], res[i,1] = mstats.kruskalwallis(Col_1.values, Col_2.values, Col_3.values)

In [ ]:
# size 18

In [ ]:
import os
df_subj = pd.DataFrame(res, columns=['H','p_value'], index=c_names)
writer = pd.ExcelWriter(os.path.join('/Users/ilyamikheev/Downloads', 'stat_amp_lat.xlsx'))
df_subj.to_excel(writer)
writer.save()

In [ ]:
import os
df_subj = pd.DataFrame(m_v, columns=['A_median','B_median','C_median','A_std','B_std','C_std'], index=c_names)
writer = pd.ExcelWriter(os.path.join('/Users/ilyamikheev/Downloads', 'median_std_amp_lat.xlsx'))
df_subj.to_excel(writer)
writer.save()